# Clean Raw Data
Cleaning Data becomes interesting depending on what kind of analysis we want to do. For this portion, since I want to visualise the data, the data typically have to be split into multiple data frames such that one numerical value always takes one row

## Import Data

In [81]:
import pandas as pd
import re
import math
import requests
import os
import json
import time
from ratelimiter import RateLimiter

In [2]:
resale_flat_prices_file_path = os.path.join('..', 'Raw Data', 'resale flat prices (08_07_2021).csv')
hdb_property_file_path = os.path.join('..', 'Raw Data', 'HDB property information.csv')
mrt_file_path = os.path.join('..', 'Raw Data', 'MRT information.csv')
bus_stops_file_path = os.path.join('..', 'Raw Data', 'Bus Stops information.csv')
bus_services_file_path = os.path.join('..', 'Raw Data', 'Bus Services information.csv')
schools_file_path = os.path.join('..', 'Raw Data', 'Schools information.csv')
supermarkets_file_path = os.path.join('..', 'Raw Data', 'supermarket information.csv')

resale_flat_prices_df = pd.read_csv(resale_flat_prices_file_path)
hdb_property_df = pd.read_csv(hdb_property_file_path)
mrt_df = pd.read_csv(mrt_file_path)
bus_stops_df = pd.read_csv(bus_stops_file_path)
bus_services_df = pd.read_csv(bus_services_file_path)
schools_df = pd.read_csv(schools_file_path)
supermarkets_df = pd.read_csv(supermarkets_file_path)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Helper functions

### Get location details based on any query (typically address)

In [3]:
geocode_url = 'https://developers.onemap.sg/commonapi/search'

@RateLimiter(max_calls=250, period=60)
def get_location_details(address, geocode_url=geocode_url):
    params = {
    'searchVal': address,
    'returnGeom': 'Y',
    'getAddrDetails': 'Y'
    }

    response = requests.get(geocode_url, params=params)

    # Checks if the one map API found any match
    if response.json()['found'] == 0:
        return None, None, None, None, None

    data = response.json()['results'][0]

    latitude = data['LATITUDE']
    longitude = data['LONGITUDE']
    postal_code = data['POSTAL']
    building = data['BUILDING']
    road_name = data['ROAD_NAME']

    # # API can only be called 250 times every min
    # time.sleep(0.25)

    return latitude, longitude, postal_code, building, road_name

### Get location details based on latitude and longitude

In [4]:
# This API requires an API Token
ONE_MAP_EMAIL = os.environ["ONE_MAP_EMAIL"]
ONE_MAP_PASSWORD = os.environ["ONE_MAP_PASSWORD"]

data = {
    'email': ONE_MAP_EMAIL,
    'password': ONE_MAP_PASSWORD
}

res = requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken', json=data)
TOKEN = res.json()['access_token']

In [5]:
reverse_geocode_url = 'https://developers.onemap.sg//privateapi/commonsvc/revgeocode'

@RateLimiter(max_calls=250, period=60)
def get_location_details_from_lat_long(latitude, longitude, geocode_url=reverse_geocode_url, TOKEN=TOKEN):
    params = {
        'location': f'{latitude},{longitude}',
        'buffer': 500,
        'token': TOKEN
    }

    response = requests.get(geocode_url, params=params)

    # Checks if the one map API found any match
    if len(response.json()['GeocodeInfo']) == 0:
        return None, None, None, None

    # Takes the first building as a proxy for postal code
    data = response.json()['GeocodeInfo'][0]

    building_name = data['BUILDINGNAME']
    block = data['BLOCK']
    road = data['ROAD']
    postal_code = data['POSTALCODE']

    # # API can only be called 250 times every min
    # time.sleep(0.25)

    return building_name, block, road, postal_code

# Clean Raw Data for Visualisation

## Clean resale_flat_prices_df

### merge with hdb_property_df

In [6]:
# hdb_property_df provides more meta data that can augment our resale data!
resale_flat_prices_merged_df = pd.merge(resale_flat_prices_df, hdb_property_df, how='left', left_on=['street_name', 'block'], right_on=['street', 'blk_no'])

In [7]:
# # Seems like there are 3755 sales (0.67%) that are not matched with the meta data
# resale_flat_prices_merged_df.info()

In [8]:
# # At this point, we can either impute the values or simply delete it
# # Since it constitutes just 0.67% of the data, and we will still have a substantial amount of data after removing them, so let's remove them
# resale_flat_prices_merged_df = resale_flat_prices_merged_df.dropna(subset=['year_completed']).reset_index(drop=True)

### remaining_lease to only be in years

> There are quite a few variations in how this is expressed. I will standardise this column to only be number of years left for the lease

In [9]:
years_regex = '(\d+) years?'

def get_remaining_lease(row):
    remaining_lease = row['remaining_lease']
    lease_commence_date = row['lease_commence_date']
    sale_date = row['month']

    # To take into account nan values. Assumption: HDBs sold have a 99 year lease
    if isinstance(remaining_lease, float) and math.isnan(remaining_lease):
        sale_date_year = int(sale_date[:4])
        remaining_lease_years = sale_date_year - lease_commence_date
        return remaining_lease_years 
    
    elif isinstance(remaining_lease, float):
        return int(remaining_lease)

    elif isinstance(remaining_lease, str):
        years_res = re.search(years_regex, remaining_lease)

        # To account for rows where this is expressed as x years y months
        if years_res is not None:
            remaining_lease_years = int(years_res.groups()[0])
            return remaining_lease_years

        # To take into account years that are strings
        else:
            return int(remaining_lease)

In [10]:
resale_flat_prices_merged_df['remaining_lease'] = resale_flat_prices_merged_df.apply(get_remaining_lease, axis=1)

### Get location details of HDB flats

In [11]:
resale_flat_prices_merged_df['address'] = resale_flat_prices_merged_df['block'] +' ' + resale_flat_prices_merged_df['street_name']

The following code took around 2 hours to run, instead of rerunning it, let's use the saved df from the result of the following chunk of code

In [12]:
address_to_coordinates_mapper = {}
address_information_dict = {
        'latitude': [],
        'longitude': [],
        'postal_code': [],
        'building': [],
        'road_name': []
}

unique_addresses = resale_flat_prices_merged_df['address'].unique()

for address in unique_addresses:
    latitude, longitude, postal_code, building, road_name = get_location_details(address)

    address_to_coordinates_mapper[address] = {
        'latitude': latitude,
        'longitude': longitude,
        'postal_code': postal_code,
        'building': building,
        'road_name': road_name
    }

    address_information_dict['latitude'].append(latitude)
    address_information_dict['longitude'].append(longitude)
    address_information_dict['postal_code'].append(postal_code)
    address_information_dict['building'].append(building)
    address_information_dict['road_name'].append(road_name)

In [13]:
# Save address_information just in case
address_information_df = pd.DataFrame(address_information_dict)

# Drop rows where latitude or longitude is None
address_information_df.dropna(subset=['latitude', 'longitude'], inplace=True)

address_information_file_path = os.path.join('..', 'Cleaned Data', 'address information.csv')
address_information_df.to_csv(address_information_file_path, index=False)

In [14]:
# address_to_coordinates_mapper_file_path = os.path.join('..', 'Helper Data', 'address_to_coordinates_mapper.json')

# with open(address_to_coordinates_mapper_file_path, 'w') as f:
#     json.dump(address_to_coordinates_mapper, f)

In [15]:
# address_to_coordinates_mapper_file_path = os.path.join('..', 'Helper Data', 'address_to_coordinates_mapper.json')

# with open(address_to_coordinates_mapper_file_path, 'r') as f:
#     address_to_coordinates_mapper = json.load(f)

In [16]:
# def get_latitude(address):
#     if address in address_to_coordinates_mapper:
#         return address_to_coordinates_mapper[address]['latitude']

#     return None

# def get_longitude(address):
#     if address in address_to_coordinates_mapper:
#         return address_to_coordinates_mapper[address]['longitude']

#     return None

# resale_flat_prices_merged_df['latitude'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['latitude'])
# resale_flat_prices_merged_df['longitude'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['longitude'])

In [17]:
resale_flat_prices_merged_df['latitude'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['latitude'])
resale_flat_prices_merged_df['longitude'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['longitude'])
resale_flat_prices_merged_df['postal_code'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['postal_code'])
resale_flat_prices_merged_df['building'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['building'])
resale_flat_prices_merged_df['road_name'] = resale_flat_prices_merged_df['address'].apply(lambda address: address_to_coordinates_mapper[address]['road_name'])

In [20]:
# Drop rows where latitude or longitude is None (Around 50,000 data points are removed)
resale_flat_prices_merged_df.dropna(subset=['latitude', 'longitude'], inplace=True)

In [76]:
# Get district of the HDB based on the postal code (Sorted by district)
postal_code_mapper = {
    '01': 1,
    '02': 1,
    '03': 1,
    '04': 1,
    '05': 1,
    '06': 1,
    '07': 2,
    '08': 2,
    '14': 3,
    '15': 3,
    '16': 3,
    '09': 4,
    '10': 4,
    '11': 5,
    '12': 5,
    '13': 5,
    '17': 6,
    '18': 7,
    '19': 7,
    '20': 8,
    '21': 8,
    '22': 9,
    '23': 9,
    '24': 10,
    '25': 10,
    '26': 10,
    '27': 10,
    '28': 11,
    '29': 11,
    '30': 11,
    '31': 12,
    '32': 12,
    '33': 12,
    '34': 13,
    '35': 13,
    '36': 13,
    '37': 13,
    '38': 14,
    '39': 14,
    '40': 14,
    '41': 14,
    '42': 15,
    '43': 15,
    '44': 15,
    '45': 15,
    '46': 16,
    '47': 16,
    '48': 16,
    '49': 17,
    '50': 17,
    '81': 17,
    '51': 18,
    '52': 18,
    '53': 19,
    '54': 19,
    '55': 19,
    '82': 19,
    '56': 20,
    '57': 20,
    '58': 21,
    '59': 21,
    '60': 22,
    '61': 22,
    '62': 22,
    '63': 22,
    '64': 22,
    '65': 23,
    '66': 23,
    '67': 23,
    '68': 23,
    '69': 24,
    '70': 24,
    '71': 24,
    '72': 25,
    '73': 25,
    '77': 26,
    '78': 26,
    '75': 27,
    '76': 27,
    '79': 28,
    '80': 28
}

In [77]:
def get_postal_district(postal_code):
    # Strangely, postal code can be nil???
    if postal_code == 'NIL':
        return None

    postal_code_first_two_num = str(postal_code)[:2]
    return postal_code_mapper[postal_code_first_two_num]

resale_flat_prices_merged_df['district'] = resale_flat_prices_merged_df['postal_code'].apply(get_postal_district)

### Drop unnecessary columns
> We will likely revisit this later :)

In [22]:
resale_flat_prices_merged_df.drop(columns=['_id_x', '_id_y', 'bldg_contract_town', 'street', 'blk_no'], inplace=True)

### Rename columns

In [23]:
resale_flat_prices_merged_df.rename(columns={'month': 'date_sold'}, inplace=True)

### Only include data from 2010 onwards for small version
> To push to Github without git lfs

In [86]:
resale_flat_prices_merged_df['date_sold'] = pd.to_datetime(resale_flat_prices_merged_df['date_sold'])
resale_flat_prices_merged_df_small = resale_flat_prices_merged_df[resale_flat_prices_merged_df['date_sold'].dt.year >= 2010].reset_index()

## Clean bus_services_df

### Get additional location data for bus stops

In [25]:
# def bus_stops_get_additional_location_data(row):
#     latitude = row['Latitude']
#     longitude = row['Longitude']

#     building_name, block, road, postal_code = get_location_details_from_lat_long(latitude, longitude)

#     row['building_name'] = building_name
#     row['block'] = block
#     row['road'] = road
#     row['postal_code'] = postal_code

#     return row

# bus_stops_df = bus_stops_df.apply(bus_stops_get_additional_location_data, axis=1)

### Merge bus stops and bus services

In [26]:
# Remove a strange BusStopCode called CTE that is not found on the bus stops data frame
bus_services_df = bus_services_df[bus_services_df['BusStopCode'].str.isnumeric()]
bus_services_df['BusStopCode'] = bus_services_df['BusStopCode'].astype('int64')

In [27]:
bus_services_merged_df = pd.merge(bus_services_df, bus_stops_df, how='left', on='BusStopCode')

## Clean mrt_df

### Get additional location data for mrt stations

In [28]:
# def mrt_get_additional_location_data(row):
#     latitude = row['Latitude']
#     longitude = row['Longitude']

#     building_name, block, road, postal_code = get_location_details_from_lat_long(latitude, longitude)

#     row['building_name'] = building_name
#     row['block'] = block
#     row['road'] = road
#     row['postal_code'] = postal_code

#     return row

# mrt_df = mrt_df.apply(mrt_get_additional_location_data, axis=1)

## Clean schools_df

### Get latitude and longitude of schools

In [29]:
def school_get_location_details(row):
    postal_code = row['postal_code']

    latitude, longitude, _, _, _ = get_location_details(postal_code)

    row['latitude'] = latitude
    row['longitude'] = longitude

    return row

schools_df = schools_df.apply(school_get_location_details, axis=1)

In [30]:
# Drop rows where latitude or longitude is None
schools_df.dropna(subset=['latitude', 'longitude'], inplace=True)

## Clean supermarket_df

### Get latitude and longitude of supermarkets

In [31]:
def supermarkets_get_lat_long(row):
    premise_address = row['premise_address']
    postal_code_start_index = premise_address.index('S(') + 2
    postal_code_end_index = postal_code_start_index + 6
    postal_code = premise_address[postal_code_start_index:postal_code_end_index]

    latitude, longitude, _, _, _ = get_location_details(postal_code)

    row['latitude'] = latitude
    row['longitude'] = longitude
    row['postal_code'] = postal_code

    return row

supermarkets_df = supermarkets_df.apply(supermarkets_get_lat_long, axis=1)

In [32]:
# Drop rows where latitude or longitude is None
supermarkets_df.dropna(subset=['latitude', 'longitude'], inplace=True)

## Clean RelaxSG.kml

### Get latitude and longitude of parks

In [33]:
# Require different libraries to read this data type
import geopandas as gpd
import fiona

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
parks_df = gpd.read_file('../Raw Data/RelaxSG.kml', driver='KML')

In [34]:
parks_df['latitude'] = parks_df['geometry'].y
parks_df['longitude'] = parks_df['geometry'].x

### Remove unnecessary columns

In [35]:
parks_df.drop(columns=['Description', 'geometry'], inplace=True)

In [58]:
# Require different libraries to read this data type
import geopandas as gpd
import fiona

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
parks_df = gpd.read_file('../Raw Data/RelaxSG.kml', driver='KML')

## Create District Map
> Source: https://github.com/lozy219/angular-singapore-district-map

In [82]:
# # Create topojson file using lozy219's district map
# singapore_districts_map_url = 'https://raw.githubusercontent.com/lozy219/angular-singapore-district-map/master/src/district.json'

# singapore_districts_map_json = requests.get(singapore_districts_map_url).json()
# singapore_topo_map = tp.Topology(singapore_districts_map_json['features'], prequantize=False)

In [84]:
# # Save district map
# singapore_topo_map_file_path = os.path.join('../Helper Data/singapore topo map.json')
# singapore_topo_map.to_json(singapore_topo_map_file_path)

## Save cleaned data frames

In [87]:
resale_flat_prices_file_path = os.path.join('..', 'Cleaned Data', 'resale flat prices.csv')
resale_flat_prices_small_file_path = os.path.join('..', 'Cleaned Data', 'resale flat prices (small).csv')
mrt_file_path = os.path.join('..', 'Cleaned Data', 'MRT information.csv')
bus_stops_file_path = os.path.join('..', 'Cleaned Data', 'Bus Stops information.csv')
bus_services_file_path = os.path.join('..', 'Cleaned Data', 'Bus Services information.csv')
schools_file_path = os.path.join('..', 'Cleaned Data', 'Schools information.csv')
supermarkets_file_path = os.path.join('..', 'Cleaned Data', 'supermarkets information.csv')
parks_file_path = os.path.join('..', 'Cleaned Data', 'parks information.csv')

resale_flat_prices_merged_df.to_csv(resale_flat_prices_file_path, index=False)
resale_flat_prices_merged_df_small.to_csv(resale_flat_prices_small_file_path, index=False)
mrt_df.to_csv(mrt_file_path, index=False)
bus_stops_df.to_csv(bus_stops_file_path, index=False)
bus_services_merged_df.to_csv(bus_services_file_path, index=False)
schools_df.to_csv(schools_file_path, index=False)
supermarkets_df.to_csv(supermarkets_file_path, index=False)
parks_df.to_csv(parks_file_path, index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8affb79e-2cc6-4186-86e6-a88dc9ffb80e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>